In [1]:
# import relevant libraries
import csv_diff
import pandas.io.json
import pandas as pd
import datetime
import sys
import csv

In [2]:
csv.field_size_limit(sys.maxsize)

131072

In [ ]:
# # temporary fix to account for new bucket names in 21.04 release
# df = pd.read_csv('data/2102_data_v1.csv')
# df = df.rename(columns = {'Bucket_1_sm':'SM_B1_Approved Drug',
#                                   'Bucket_2_sm':'SM_B2_Advanced Clinical',
#                                   'Bucket_3_sm':'SM_B3_Phase 1 Clinical',
#                                   'Bucket_4_sm':'SM_B4_Structure with Ligand',
#                                   'Bucket_5_sm':'SM_B5_High-Quality Ligand',
#                                   'Bucket_6_sm':'SM_B6_High-Quality Pocket',
#                                   'Bucket_7_sm':'SM_B7_Med-Quality Pocket',
#                                   'Bucket_8_sm':'SM_B8_Druggable Family',
#                                   'Bucket_1_ab':'AB_B1_Approved Drug',
#                                   'Bucket_2_ab':'AB_B2_Advanced Clinical',
#                                   'Bucket_3_ab':'AB_B3_Phase 1 Clinical',
#                                   'Bucket_4_ab':'AB_B4_UniProt loc high conf',
#                                   'Bucket_5_ab':'AB_B5_GO CC high conf',
#                                   'Bucket_6_ab':'AB_B6_UniProt loc med conf',
#                                   'Bucket_7_ab':'AB_B7_UniProt SigP or TMHMM',
#                                   'Bucket_8_ab':'AB_B8_GO CC med conf',
#                                   'Bucket_9_ab':'AB_B9_Human Protein Atlas loc',
#                                   'Bucket_1_PROTAC':'PR_B1_Approved Drug',
#                                   'Bucket_2_PROTAC':'PR_B2_Advanced Clinical',
#                                   'Bucket_3_PROTAC':'PR_B3_Phase 1 Clinical',
#                                   'Bucket_4_PROTAC':'PR_B4_Literature',
#                                   'Bucket_5_PROTAC':'PR_B5_UniProt Ubiquitination',
#                                   'Bucket_6_PROTAC':'PR_B6_Database Ubiquitination',
#                                   'Bucket_7_PROTAC':'PR_B7_Half-life Data',
#                                   'Bucket_8_PROTAC':'PR_B8_Small Molecule Binder',
#                                   'PROTAC_location_Bucket':'PR_locB_Location Score',
#                                   'Bucket_1_othercl':'OC_B1_Approved Drug',
#                                   'Bucket_2_othercl':'OC_B2_Advanced Clinical',
#                                   'Bucket_3_othercl':'OC_B3_Phase 1 Clinical'
#                                  })
# df.to_csv('data/2102_data_mapped.csv')

In [4]:
# load current and previous release data
data_previous_release = csv_diff.load_csv(open('data/21_06_data_v1.csv'), key='ensembl_gene_id')
data_current_release = csv_diff.load_csv(open('data/21_09_data_v1.csv'), key='ensembl_gene_id') 

print("Records in the previous release data = %i" % len(data_previous_release))
print("Records in the current release data = %i" % len(data_current_release))

difference_in_release_data = len(data_current_release) - len(data_previous_release) 
print("Difference between the number of rows in the two release data files = %i" % (difference_in_release_data))

Records in the previous release data = 19498
Records in the current release data = 19486
Difference between the number of rows in the two release data files = -12


In [5]:
# run CSV diff on data files (output is JSON as specified in the csv_diff docs)
tractability_data_csv_diff_results = csv_diff.compare(data_previous_release, data_current_release)

In [6]:
# convert JSON into DataFrame and save as CSV
normalised_csv_diff_data = pandas.io.json.json_normalize(tractability_data_csv_diff_results['changed'])
tractability_csv_diff_df = pd.DataFrame.from_dict(normalised_csv_diff_data, orient='columns')
print('Generated tractability diff file with %i records changed' % (len(tractability_csv_diff_df)))

Generated tractability diff file with 550 records changed


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [7]:
# generate and save report report
today_date = datetime.datetime.now().strftime('%Y-%m-%d')
report_directory = 'reports/'
report_filename = today_date + '_' + 'tractability_data_csv_diff_report.csv'
tractability_csv_diff_df.to_csv(report_directory + report_filename, index=False)

In [8]:
tractability_csv_diff_df.columns

Index(['key', 'changes.OC_B2_Advanced Clinical',
       'changes.OC_B3_Phase 1 Clinical', 'changes.AB_B2_Advanced Clinical',
       'changes.OC_B1_Approved Drug', 'changes.SM_B1_Approved Drug',
       'changes.SM_B2_Advanced Clinical',
       'changes.AB_B4_UniProt loc high conf',
       'changes.AB_B6_UniProt loc med conf',
       'changes.SM_B4_Structure with Ligand',
       'changes.PR_B8_Small Molecule Binder', 'changes.AB_B8_GO CC med conf',
       'changes.accession', 'changes.SM_B5_High-Quality Ligand',
       'changes.AB_B7_UniProt SigP or TMHMM', 'changes.AB_B3_Phase 1 Clinical',
       'changes.SM_B3_Phase 1 Clinical',
       'changes.PR_B6_Database Ubiquitination',
       'changes.PR_B5_UniProt Ubiquitination', 'changes.symbol',
       'changes.PR_B7_Half-life Data', 'changes.AB_B1_Approved Drug',
       'changes.AB_B5_GO CC high conf'],
      dtype='object')

In [9]:
all_clinical_precedence_buckets = [
    'changes.SM_B1_Approved Drug',
    'changes.SM_B2_Advanced Clinical',
    'changes.SM_B3_Phase 1 Clinical',
    'changes.AB_B1_Approved Drug',
    'changes.AB_B2_Advanced Clinical',
    'changes.AB_B3_Phase 1 Clinical',
    'changes.OC_B1_Approved Drug',
    'changes.OC_B2_Advanced Clinical',
    'changes.OC_B3_Phase 1 Clinical',
    'changes.PR_B1_Approved Drug'
    'changes.PR_B2_Advanced Clinical',
    'changes.PR_B3_Phase 1 Clinical'
]

clinical_predence_buckets_in_csv_diff_file = ['key'] 

for bucket in all_clinical_precedence_buckets:
    if bucket in tractability_csv_diff_df.columns:
        clinical_predence_buckets_in_csv_diff_file.append(bucket)

print(clinical_predence_buckets_in_csv_diff_file)

['key', 'changes.SM_B1_Approved Drug', 'changes.SM_B2_Advanced Clinical', 'changes.SM_B3_Phase 1 Clinical', 'changes.AB_B1_Approved Drug', 'changes.AB_B2_Advanced Clinical', 'changes.AB_B3_Phase 1 Clinical', 'changes.OC_B1_Approved Drug', 'changes.OC_B2_Advanced Clinical', 'changes.OC_B3_Phase 1 Clinical']


In [10]:
tractability_clin_precedence_df = tractability_csv_diff_df.filter(clinical_predence_buckets_in_csv_diff_file)
tractability_clin_precedence_df.set_index('key', inplace=True, drop=True)
tractability_clin_precedence_df.head()

,changes.SM_B1_Approved Drug,changes.SM_B2_Advanced Clinical,changes.SM_B3_Phase 1 Clinical,changes.AB_B1_Approved Drug,changes.AB_B2_Advanced Clinical,changes.AB_B3_Phase 1 Clinical,changes.OC_B1_Approved Drug,changes.OC_B2_Advanced Clinical,changes.OC_B3_Phase 1 Clinical
key,,,,,,,,,
ENSG00000146648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0, 1]",NaN
ENSG00000141736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0, 1]",NaN
ENSG00000198900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0, 1]"
ENSG00000065361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0, 1]",NaN
ENSG00000066468,NaN,NaN,NaN,NaN,"[0, 1]",NaN,NaN,NaN,NaN


In [11]:
tractability_clin_precedence_df.dropna(how="all", inplace=True)
tractability_clin_precedence_df.head()

,changes.SM_B1_Approved Drug,changes.SM_B2_Advanced Clinical,changes.SM_B3_Phase 1 Clinical,changes.AB_B1_Approved Drug,changes.AB_B2_Advanced Clinical,changes.AB_B3_Phase 1 Clinical,changes.OC_B1_Approved Drug,changes.OC_B2_Advanced Clinical,changes.OC_B3_Phase 1 Clinical
key,,,,,,,,,
ENSG00000146648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0, 1]",NaN
ENSG00000141736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0, 1]",NaN
ENSG00000198900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0, 1]"
ENSG00000065361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0, 1]",NaN
ENSG00000066468,NaN,NaN,NaN,NaN,"[0, 1]",NaN,NaN,NaN,NaN


In [12]:
print(len(tractability_clin_precedence_df))

257


In [13]:
today_date = datetime.datetime.now().strftime('%Y-%m-%d')
report_directory = 'reports/'
report_filename = today_date + '_' + 'tractability_data_clin_prec_only_csv_diff_report.csv'
tractability_clin_precedence_df.to_csv(report_directory + report_filename, index=True)